# The Iconic Data Science Challenge

![the_iconic](https://www.lifehacker.com.au/coupons/vfiles/55-27fd5e0f09d2a4389c956496f4e35a6b.png)

## Table of Content

1. Scenario
2. Data Inspection
3. Data Cleaning
4. Feature Engineering
5. Feature Selection
6. Gender Inference
7. Gender Classification
8. Conclusion

## 01 Scenario

The Iconic collects a lot of useful information from its users while respecting their privacy. Because of this though, gender is not explicitely asked from its customers but rather inferred with algorithms or by incentivizing customers to provide such information. The motivation for wanting this piece of information is clear, in order to provide consumers with the best experience possible, and to put in from of them most relevant items possible, this piece of information makes such a task a much easier one.

In this notebook I tackle 5 tasks using data from customers at The Iconic, data cleaning, feature engineering, feature selection, gender inference and classification. The main goal of this notebook is to provide a document that takes its user from data cleaning to prediction rather that a system for internal use that could go into production, those steps steps are different and thus not emphazised in this notebook.

## 02 Data Inspection

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os, json, time


from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import umap
import joblib

from sklearn.metrics import plot_confusion_matrix
from sklearn.model_selection import GridSearchCV
import xgboost as xgb

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold, cross_val_score, train_test_split

pd.options.display.max_columns = None
pd.options.display.max_rows = None

%matplotlib inline

The data provided comes in JSON format and thus, we will read it line by line, add it to a list, and create our dataframe from that list of dictionaries.

In [ ]:
path = 'data'
file = os.path.join(path, 'raw', 'data.json')

In [ ]:
data_list = []

for line in open(file, 'r'):
    data_list.append(json.loads(line))

In [ ]:
df = pd.DataFrame(data_list)

In [ ]:
# df.head()

In [ ]:
# df.info(memory_usage='deep')

Check for duplicates.

In [ ]:
# df.duplicated().sum()

Check if individual items add up to the items variables, if so, these will likely be correlated.

In [ ]:
# (df['female_items'] + df['male_items'] + df['unisex_items'] - df['items']).sum()

Inspect the unique values of all features. Go through each one by one.

In [ ]:
# for col in df.columns:
#     print(col)
#     print(f"Unique # of values: {df[col].nunique()}")
#     print(df[col].unique())
#     print()
#     print('-' * 50)
#     print()

Is the `sacc_items` variable unique or is it related to the gender sports variables?

In [ ]:
# df.sacc_items.value_counts()

In [ ]:
# (df.mspt_items + df.wspt_items - df.sacc_items).sum()

Are there a lot of work orders or can this variable be recoded as a dummy that says if the customer has order an item to their work.

In [ ]:
# df.work_orders.value_counts(normalize=True)

Do all gender items add up to the items column?

In [ ]:
# (df.female_items + df.male_items + df.unisex_items - df['items']).sum()

Is there a particular order type or payment that dominates all transactions?

In [ ]:
# (df.cc_payments.sum() + df.afterpay_payments.sum() + df.paypal_payments.sum() + df.apple_payments.sum() - df['orders'].sum())

Let's look at the distinction between payments and orders.

In [ ]:
# payments = (df.cc_payments + df.afterpay_payments + df.paypal_payments + df.apple_payments)
# orders = (df.msite_orders + df.desktop_orders + df.android_orders + df.ios_orders + df.other_device_orders)
# how_orders = (df.work_orders + df.home_orders + df.parcelpoint_orders + df.other_collection_orders)
# all_orders = pd.concat([payments, orders, how_orders], axis=1)
# all_orders.columns = ['payments', 'orders', 'how_orders']
# all_orders.head(10)

In [ ]:
# (all_orders.iloc[:, 1] - all_orders.iloc[:, 2]).sum()

Examine the footware variable.

In [ ]:
# df.wftw_items.value_counts(normalize=True)

In [ ]:
# df.mftw_items.value_counts(normalize=True)

In [ ]:
# df.wftw_items.sum() / (df.mftw_items.sum() + df.wftw_items.sum())

It seems like women order more shoes than man.

Let's look at the apparel now.

In [ ]:
# (df.wapp_items.sum() / (df.mapp_items.sum() + df.wapp_items.sum())).sum()

Are Curvy Items a common purchase? Does the variable need to be rocoded to be a better predictor.

In [ ]:
# df.curvy_items.value_counts(normalize=True)

Barely any curvy items purchase. Nonetheless, the few that have been purchase might provide a good signal in a dummy variable towards our end goal.

Do people cancel often.

In [ ]:
# df.cancels.value_counts().plot(kind='bar');

In [ ]:
# (df.loc[df.cancels == 0, 'revenue'] == 0).sum()

In [ ]:
# (df.loc[df.revenue == 0, 'cancels'] == 0).sum()

Barely any cancellations. This variable might be more useful as a dummy rather than a continous one.

Returns could provide a good indicator as to which gender is buying and returning items, but because of the skewness of this variable, we might be better served by creating a dummy for it called, `has_returned_item`.

In [ ]:
# df.returns.describe()

Vouchers are important and may incentivize customers to purchase more items when they are about to use the voucher.

In [ ]:
# df.vouchers.value_counts()

Let's check Shipping Addresses and devices for any inconsistencies or outliers.

In [ ]:
# df.shipping_addresses.value_counts(normalize=True)

In [ ]:
# df.devices.value_counts()

Devices might be more useful as a categorical variable with for each label.

Diff addresses is already a binary variable so we will keep it as is.

In [ ]:
# df.different_addresses.value_counts()

Let's examine the `days_since_first_order` variable which seemed a bit odd at first glance when evaliating the unique values.

In [ ]:
# (df['days_since_first_order'] < df['days_since_last_order']).sum()

In [ ]:
# df[['days_since_first_order', 'days_since_last_order']].head()

The variable seems to be in the wrong unit. It doesn't seem plausible that a client made its first purchase 80+ years ago while The Iconic has been in business for much less than that. Let's check different time units such as minutes or hours.

In [ ]:
# (df['days_since_last_order'] / 24).head()

Dviding by 24 hours seem to be the best course of action here.

`average_discount_used` seems higher than revenue in some instances, could it be due to a discount offer or a voucher used by the customer?

In [ ]:
# df[['average_discount_used', 'average_discount_onoffer', 'revenue']].head(10)

Let's looks at the missing values.

In [ ]:
# df.isna().sum() / df.shape[0] * 100

`coupon_discount_applied` has a few missing values but we can't just drop these rows. Let's examine the revenue made from and female items sold to all of our customers where this variable is missing.

In [ ]:
# df.groupby(df['coupon_discount_applied'].isna())[['female_items', 'revenue']].describe().T

In [ ]:
# df.groupby(df['coupon_discount_applied'].isna())['revenue'].mean().plot(kind='bar');

In addition to `coupon_discount_applied`, `redpen_discount_used` was not in the dictionary providedd. Let's examine them alongside the other coupon vars to see if we get a better idea as to what might be happening.

In [ ]:
# df.loc[df.coupon_discount_applied.notna(), ['orders', 'redpen_discount_used', 'revenue', 
#                                             'average_discount_onoffer', 'average_discount_used', 'coupon_discount_applied']].head()

Finally let's look at the correlation Matrix of our numerical variables to see which vars are highly correlated with one another.

In [ ]:
# numerical_cols = [i for i in df.columns if str(df[i].dtype) != 'object']

# plt.figure(figsize=(20, 10))
# # take out the annotate parameter to see only the colors
# sns.heatmap(df[numerical_cols].corr(), annot=True, fmt=".1f", cmap='viridis')
# # plt.xticks(rotation=75) # uncomment this to rotate the x axis
# plt.show();

After some thorough inspection of the data, here are some of the common issues that need to be fixed.

Cleaning Tasks

1. There were 249 duplicates in the dataset by all variables and by the customer id alone. These will be removed.
2. The dataset contains additional variables that were not in the dictionary provided, namely, `coupon_discount_applied` and `redpen_discount_used`. The latter does not add up completely to the `avg_discount_onoffer` or the discount used, nor does it match the proportional discount of the former. Until further investigation of the data generating process for these two, both will be dropped or ignored in the feature engineering and modeling phases.
4. The column `days_since_last_order` needs to be either removed as it says that some clients have taken 90+ years to make an order or, it seems that adjusting it from hours to days (divide by 24) brings it to a reasonable level.
5. The variable containing the average discount used seems to contain percentage rather than such large numbers. In many instances, this value is much larger than the revenue. Dividing this variable by 10,000 brings it down to a percentage that follows closely discount on offer. As this makes the most sense, the `average_discount_used` variable will be divided by 10,000.

Feature engineering tasks

1. The "is newsletter subscriber" column is the only categorical variable that is not numerical and it will be encoded as a dummy variable.
2. Sports accessories is independent from the womens and men sport items. It will be converted into a dummy for those who purchased an accessory and those who didn't.
3. Sports, footware, apparel and regular items will all be split into a variable representing the percentage of the gender the item purchased belong to.
4. Work orders will become a dummy variable and all orders will be reduced to a percentable of home versus other.
5. Destop order will be represented as a percentage against all other orders of that kind.
6. Curvy items could add some signal towards gender but its proportion is quite low. Because of this, it will be converted into a dummy.
7. Cancellations will be represented by the dummy `has_cancelled`.
8. Devices will be represented as a binary variable called `multiple_devices`.
9. Days as a customer will be represented as average days between purchases.

## 04 Data Cleaning

Get rid of duplicates.

In [ ]:
df = df.drop_duplicates(subset='customer_id')

Recode `days_since_last_order`.

In [ ]:
df['days_since_last_order'] = (df['days_since_last_order'] / 24)

Divide by `average_discount_used` by 10000.

In [ ]:
df['average_discount_used'] = (df['average_discount_used'] / 10000)

More than half of the missing values are missing where there is no coupon on offer or available. In addition, the variable is highly skewed and since the median and the mode are the same we will use these 2 to fill in the missing values.

In [ ]:
df['coupon_discount_applied'].fillna(0, inplace=True)

In [ ]:
df.isna().sum() / df.shape[0] * 100

Let's save the cleaned dataset for later use.

In [ ]:
df.to_csv(os.path.join(path, 'clean', 'clean_data.csv'), index=False)

## 05 Feature Engineering

We will first create all of the binary variables previously discussed.

In [ ]:
df = pd.get_dummies(df, columns=['is_newsletter_subscriber'], drop_first=True)
df['curvy_item_dummy'] = df.curvy_items > 0
df['work_order_dummy'] = df.work_orders > 0
df['has_cancelled'] = df.cancels > 0
df['multiple_devices'] = df.devices > 1

Let's now create a function to help us create the percentage columns for the items and orders placed.

In [ ]:
def get_pct(data, col, cols_to_add):
    df_temp = data.copy()
    df_temp['pct_' + col] = (df_temp[col] / df_temp[cols_to_add].sum(axis=1))
    return df_temp

In [ ]:
df = get_pct(df, 'female_items', ['male_items', 'female_items', 'unisex_items'])
df = get_pct(df, 'wapp_items', ['wapp_items', 'mapp_items'])
df = get_pct(df, 'wacc_items', ['wacc_items', 'macc_items'])
df = get_pct(df, 'wspt_items', ['wspt_items', 'mspt_items'])
df = get_pct(df, 'cc_payments', ['cc_payments', 'paypal_payments', 'afterpay_payments', 'apple_payments'])
df = get_pct(df, 'desktop_orders', ['msite_orders', 'desktop_orders', 'android_orders', 'ios_orders', 'other_device_orders'])
df = get_pct(df, 'home_orders', ['home_orders', 'work_orders', 'parcelpoint_orders', 'other_collection_orders'])

To create the average days between orders, we will subtract the first from last order, and then divide by all orders.

In [ ]:
df['avg_days_to_purcahse'] = ((df.days_since_first_order  - df.days_since_last_order) / df.orders)

Some items will have been divided by 0 and are now represented as NaN. we will fill these up with 0 again.

In [ ]:
df.fillna(0, inplace=True)

## 06 Feature Selection

Since we will first cluster the data into groups that make sense to use in place of gender, we will not use a regularazation method here to tell us which ones to pick. We will pick the ones identified above through the analysis and the correlation matrix.

In [ ]:
numerical_cols = ['orders', 'revenue', 'returns', 'vouchers', 'average_discount_used',
                  'shipping_addresses', 'unisex_items', 'other_device_orders',
                  'average_discount_onoffer', 'pct_female_items', 'pct_wapp_items',
                  'pct_wacc_items', 'pct_wspt_items', 'pct_cc_payments',
                  'pct_desktop_orders', 'pct_home_orders', 'avg_days_to_purcahse']

cat_cols =  ['is_newsletter_subscriber_Y', 'curvy_item_dummy', 'different_addresses', 'work_order_dummy', 'has_cancelled', 'multiple_devices']

all_cols = numerical_cols + cat_cols

## 07 Gender Inference

We will infer gender with an usupervised algorithm, k-means, and using only the numerical variables. We will do so by

1. Scaling the features by subtracting the mean and dividing by the standard deviation. Note that this assumes that the underlying distribution of our numerical vars is normal.
2. Reduce the dimensions to 2 with UMAP (short for universal manifolm approximation and projection.
3. Custer our 2 dimensions into 2 clusters, males and females, using k-means.
4. Evalueate our result with the silhouette score for our predicted labels.

Note that kmeans assumes that the shape of the data is spherical and thus, it draws a circle a s best as it can.

In [ ]:
preprocessing = Pipeline([
    ('scaler', StandardScaler()),
    ('umap', umap.UMAP(n_neighbors=40, n_components=2, metric='manhattan', unique=False, random_state=42))
])
model = Pipeline([('km', KMeans(n_clusters=2, n_init=50, max_iter=500, random_state=42))])

pipe = Pipeline([('preprocessing', preprocessing), ('model', model)])

Note, it will take about a minute for the pipeline to finish fitting the data.

In [ ]:
%%time

pipe.fit(df[numerical_cols])

In [ ]:
%%time

preprocessed_data = pipe["preprocessing"].transform(df[numerical_cols])

preds = pipe["model"]["km"].labels_

silhouette_score(preprocessed_data, preds)

In [ ]:
pd.Series(preds).value_counts(normalize=True).plot(kind='bar');

In [ ]:
plt.scatter(preprocessed_data[:, 0], preprocessed_data[:, 1], c=preds, cmap='viridis');

In [ ]:
df['gender'] = preds

## 08 Gender Classification

Let's now get to classifying the gender. We will start by splitting the dataset into training and test set while specifying that our target variable requires stratification.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df.loc[:, all_cols].copy(), df['gender'].copy(), random_state=42, stratify=df['gender'])

Instantiate model.

In [ ]:
clf_xgb = xgb.XGBClassifier(objective="binary:logistic", seed=42)

First model before hyperparameter tunning.

In [ ]:
# clf_xgb.fit(X_train, y_train, verbose=True, early_stopping_rounds = 10, eval_metric='aucpr', eval_set=[(X_test, y_test)])

Hyperparameter tuning.

In [ ]:
# Note this cell can take quite a bit to run
# param_grid = {'max_depth': [4, 5, 6],
#               'learning_rate': [0.1, 0.05, 0.01],
#               'gamma': [0, 0.25, 1],
#               'reg_lambda': [0, 1.0, 10],
#               'scale_pos_weight': [1, 3, 5]}

# optimal_params = GridSearchCV(estimator=xgb.XGBClassifier(objective="binary:logistic", seed=42, subsample=0.8, colsample_bytree=0.5),
#                              param_grid=param_grid, scoring='roc_auc', verbose=0, n_jobs=-1, cv=5)


# optimal_params.fit(X_train, y_train, verbose=False, early_stopping_rounds = 10, eval_metric='auc', eval_set=[(X_test, y_test)])

Extract the best parameters.

In [ ]:
# print(optimal_params.best_params_)

Build the model with our new parameters.

In [ ]:
clf_xgb = xgb.XGBClassifier(objective="binary:logistic", seed=42, gamma=0, learning_rate=0.1, max_depth=4, reg_lambda=0, scale_pos_weight=1)

Train it.

In [ ]:
clf_xgb.fit(X_train, y_train, verbose=True, early_stopping_rounds = 10, eval_metric='aucpr', eval_set=[(X_test, y_test)])

Extract the features and their importance.

In [ ]:
feat_importance = pd.DataFrame(clf_xgb.feature_importances_, 
                               index=all_cols, 
                               columns=['Importance']).sort_values(['Importance'], ascending=False)

feat_importance['Index'] = range(feat_importance.shape[0])
feat_importance_cut = feat_importance[feat_importance['Importance'] > 0.01]

Plot the features.

In [ ]:
plt.figure(figsize=(15, 5))
sns.pointplot(x='Index', y='Importance', data=feat_importance_cut, linestyles='')
plt.xlabel(xlabel='')

for i, ind in enumerate(feat_importance_cut.index):
    x = feat_importance.loc[ind, 'Index']
    y = feat_importance.loc[ind, 'Importance']
    plt.text(x+0.08, y, ind, fontsize=9)

## 09 Conclusion

While the output of the classification model provided a high accuracy, the labels it used as its source of truth for training and validation came form a completely unsupervised process with a lot of room for improvement. For example, Universal Manifold Approximation and Projection is an excellent algorithm for dimensionality reduction and pure exploratory data analysis. That said, other dimmensionality reduction techniques, such as non-negative factorization matrix or, other clustering algorithms that do not expect the parameter K but that provide noise in place of the wrong assignment of a label, might have been better suited for the gender inference task. Our k-means only achieved a silhouette score of ~67 and this roughly tells us the measure by which our groups are clorely related to themeselves and closely unrelated to the other group(s).

Nevertheless, a proof of concept such as the one created for this task can be beneficial for the operations at The Iconic in several ways. For example, while gender was not a clear cut replacement for the source of truth, segmenting consumers based of their purchasing preference, whether based on male or female clothing purchases, or sporty or accessory-full ones, these segments can help tailor marketing strategies more effectively and customize the experience of our customers further.